<a href="https://colab.research.google.com/github/Dadmatech/Persian-spell-checkers-comparison/blob/main/Persian_Spell_checkers_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir='rtl'>

# ارزیابی اصلاح گر املایی فارسی


امروزه طیف وسیعی از کاربران فارسی‌زبان اعم از دانشجویان، محققین و تولیدکنندگان محتوا با اصلاح‌گرهای املایی سروکار دارند. اصلاح‌گر املایی چنان‌چه عملکرد مطلوبی ارائه دهد،  هم فرایند نگارش را تسریع نموده و هم کیفیت متن نهایی را ارتقا می‌بخشد. به‌رغم معرفی و توسعه چندین اصلاح‌گر املایی مختص به فارسی در سال‌های اخیر، اطلاعات جامع و قابل‌اتکایی از کیفیت هریک و به‌خصوص در قیاس با یکدیگر در دست نیست. این امر ما را برآن داشت تا با جمع‌آوری و تهیه دادگان ارزیابی استاندارد و متنوع، اصلاح‌گرهای املایی مشهور موجود در زبان فارسی را در بوته آزمایش قرار داده و نتایج را به‌تفکیک و با شرح توضیحات لازم، به‌صورت عمومی منتشر سازیم. امید داریم ماحصل این تلاش، هم کاربران را در گزینش اصلاح‌گر املایی دلخواه یاری رساند و هم توسعه‌دهندگان این دست محصولات را بیش از پیش، نسبت به کیفیت محصول خود و محصولات مشابه آگاهی بخشد.



## دادگان ارزیابی


بررسی‌های صورت‌گرفته  در راستای یافتن دادگان ارزیابی مناسب، آخرسر سه دادگان ذره‌بین، PerSpellData  و شرق را پیش‌روی ما نهاد. باتوجه به ضعف‌های هریک از این دادگان که در ادامه بیان خواهد شد، دو مجموعه دادگان جدید نیز توسط تیم ما ساخته و به‌کارگرفته شد. جزئیات آماری هریک از این دادگان، در جدول زیر قابل مشاهده است.

- **دادگان ذره‌بین**: مجموعه‌ای از ۱۰۳۳ زوج عبارت صحیح و عبارت دارای خطا از سرچ‌های کاربران سامانه ذره‌بین همراه اول

- **دادگان ارزیابی PerSpellData**: مجموعه‌داده‌ی ارزیابی معرفی شده در مقاله‌ی PerSpellData * شامل ۱۱۲۷ زوج عبارت صحیح و عبارت دارای خطا


> PerSpellData: An Exhaustive Parallel Spell Dataset For Persian, 2021

- **دادگان شرق** -
شامل ۲۲۳ زوج عبارت صحیح و عبارت دارای خطا
تهیه شده توسط دکتر ممتازی

- **مجموعه دادگان نویسه**
: یکی از روش‌های نوین در ساخت زوج دادگان املایی، روش خودکار و مبتنی بر اعمال نویز است. بر اساس این رویکرد، دادگان نویسه تولید شده و در این ارزیابی نیز مورد استفاده قرار گرفته است. در  ساخت این دادگان کوشیده‌ایم، انواع و اقسام خطاهای محتمل املایی را پوشش دهیم. علاوه بر این، عوامل اثرگذاری هم‌چون جایگاه حروف در صفحه کلیدهای فارسی، حروف هم‌آوا، حروف هم‌شکل و غلط‌های مصطلح را نیز تا حد امکان در ساخت دادگان دخیل نموده‌ایم. نکته پایانی هم آن‌که ماده خام این دادگان، آخرین خبرهای خزش‌شده از صفحات خبری فارسی بوده تا احتمال هم‌پوشانی دادگان ارزیابی با دادگان آموزش اصلاح‌گرهای املایی به حداقل برسد. 

>  ۱.**دادگانی از جملات متن‌های خبری**

> ۲.**دادگان عنوان خبرها**

> ۳. **دادگان عنوان خبر - ۵۳۹**
دادگان کوچک‌تری که از ۵۳۹ رکورد اول دادگان عنوان‌ خبرهای درست‌ست تعریف شده است تا  ارزیابی آن دسته از اصلاح‌گرهای املایی که نیاز به اعمال دستی دارند تسهیل شود.

اطلاعات کلی مجموعه دادگان یاد شده:

| نام دادگان‌ | تعداد جمله‌ها | میانگین تعداد کلمات در جمله | تعداد خطا | میانگین خطا در جمله | دانلود |
| -- | -- | -- | -- | -- | -- |
| **دادگان  ذره‌بین** | ۱۰۳۳ | ۳.۵۴ | ۱۴۰۷ | ۱.۳۶| [لینک](https://drive.google.com/drive/folders/1PofK9ASp6TJ7Q-DZKUmPfLd8zabWHKFo?usp=sharing)|
| **دادگان PerSpellData** | ۱۱۲۷ | ۱۲.۹ | ۱۱۵۷ | ۱.۰۲ | [لینک](https://drive.google.com/drive/folders/1v-Zz5D1SceFumVHEz49P-l4-1yfysiyx?usp=sharing) |
| **دادگان شرق** | ۲۲۳ | ۸.۵۱ | ۲۲۲ | ۰.۹۹ | [لینک](https://drive.google.com/drive/folders/1n3STWIfY7bfTek7OMJPuzJI51op3EpfL?usp=sharing) |
| **دادگان نویسه - متن خبر** | ۴۵۱ | ۲۴.۷۶ | ۲۳۰۵ | ۵.۱۱ | [لینک گیت‌هاب](https://github.com/Dadmatech/Nevise-Dataset/tree/main/nevise-news-451) |
|  **دادگان نویسه - عنوان خبر -  ۵۳۹**|  ۵۳۹ | ۱۰.۰۲ | ۵۱۰ | ۰.۹۴ | [لینک گیت‌هاب](https://github.com/Dadmatech/Nevise-Dataset/tree/main/nevise-news-title-539) |
| **دادگان نویسه - عنوان خبر** | ۱۹۴۲۱ | ۱۰.۲۱ | ۱۸۳۶۸ | ۰.۹۴ | [لینک گیت‌هاب](https://github.com/Dadmatech/Nevise-Dataset/tree/main/nevise-news-title-all) |


### چالش‌های مجموعه‌ها

- ذره‌بین: این دادگان منحصر به جست‌وجوهای کاربران بوده و بنابراین برای فرم‌های دیگری از متن مانند متون بلند  محک مناسبی به‌شمار نمی‌رود.

- دادگان شرق: حجم اندک این دادگان، نقطه‌ضعف اصلی آن محسوب می‌شود.

- پراسپل‌دیتا-تست‌ست: این دادگان، اگرچه غلط‌های مصطلح را  مدنظر قرار داده اما از تنوع بسیار کمی برخوردار است. برای مثال در  ۵ رکورد اول این دادگان خطای یکتا وجود دارد.

| ردیف | جمله |
| -- | -- |
| ۱ | مرادی با اشاره به آبات قرآن که خداوند می‌فرماید  |
| ۲ | قرآن راه سعادت آبات قرآن و احادیث مرتبط آیه‌های قرآنی و فرهنگ مسلمانی و سعادت در خانه بسیار قرآن خوانده شود |
| ۳‌ | و حتی آبات قرآن را هم می‌دونستم |
| ۴ | همان بیتی که آبات نور الهی در آن نازل گردیده |
| ۵‌ | و خداوند تعلل و کوتاهی و سهل انگاری در این زمینه را طبق صریح آبات قرآن و متون دینی صحیح ذیربط از ما نخواهد بخشید |




## معیارهای ارزیابی

مرحله‌ی بعدی معرفی معیارهایی برای سنجش است. معمولا نرخ تشخیص خطا و نرخ اصلاح واژه‌‌ها می‌توانند مهمترین معیار باشند اما نرخ واژه‌های صحیحی که غلط تشخیص داده‌می‌شوند نیز از اهمیت بالایی برخوردار است. به همین دلیل معیار precision با توجه به مسئله تعریف می‌شود. 

 نحوه‌ی محاسبه‌ی معیارها:


> **wrongs**: تعداد غلط‌های نوشته

> **detected**: تعداد غلط‌های کشف‌شده توسط الگوریتم

> **c2c**: تعداد کلمات صحیحی که دست‌نخورده باقی‌ماندند

> **w2c**: تعداد غلط‌هایی که تصحیح شدند

> **c2w**: تعداد کلمات صحیحی که تخریب شدند

> **w2w**: تعداد کلمات غلطی که با کلمه‌ی اشتباه دیگری جایگزین شدند


-  نرخ کشف خطا
 (Recall)

$$\frac{detected}{wrongs}$$

- نرخ صحت کشف خطاها
(Precision)

$$\frac{w2c + w2w}{c2w + w2c + w2w}$$



- نرخ اصلاح واژه‌های خطا

$$\frac{w2c}{wrongs}$$

- نرخ تخریب واژه‌های صحیح

$$\frac{c2w}{w2c + c2c}$$



## نحوه‌ی ارزیابی اصلاح‌گرهای املایی
 
- **اصلاح‌گر املایی گوگل** در پیشنهاد‌های google doc برای جایگزینی واژه‌‌ها دیده‌ می‌شود. متن به‌دست آمده  از اعمال دستی این تغییرات به عنوان خروجی اصلاح‌گر املایی گوگل درنظر گرفته شد. چون فرایند استفاده از این اصلاح‌گر زمان‌بر است تنها برای ارزیابی دادگان‌هایی با حجم کوچک استفاده می‌شود.

- **ویراست‌من** برای ارزیابی تنها به کمک افزونه‌ای در MS Office Word در دسترس است و با اعمال دستی می‌توان کلمات را تصحیح کرد. به دلیل این محدودیت ارزیابی فقط روی مجموعه‌داده‌های کوچک امکان‌پذیر است. ویراست‌من ممکن است برای هر کلمه‌ای که اشتباه تشخیص دهد چندین پیشنهاد ارائه دهد که در این ارزیابی اولین گزینه انتخاب می‌شود بدون توجه به این که کدام‌یک از پیشنهاد‌‌ها می‌تواند پیشنهاد مناسب‌تری باشد.

- **پاک‌نویس**: برای ارزیابی از API پاک‌نویس که مبتنی بر توکن است استفاده کردیم که توکن توسط تیم پاک‌نویس در اختیار شرکت دادماتک قرارگرفته است.


- **نویسه** دمو اصلاح‌گر نویسه نسخه‌ی ۱ و ۲ در [این آدرس](https://dadmatech.ir/#/products/SpellChecker) و در دو تب جدا قرار دارد.



## ملاحظات ارزیابی


- یکی از چالش‌های ارزیابی اصلاح‌گر املایی برای متن فارسی، نوشته‌شدن واژه‌ها به اشکال مختلف است که این تنوع در استفاده یا عدم استفاده از نیم‌فاصله و فاصله یا استفاده از برخی کاراکتر‌های خاص مثل همزه و تنوین پیش‌می‌آید. اما چون اشکالاتی حتی در دیتای صحیح دیده می‌شود و تشخیص درستی آن گاهی تنها به کمک زبان‌شناس ممکن است، تا حد امکان خطاهای مربوط به این تفاوت برای سامانه‌ها درنظر گرفته نمی‌شود. 

- برخی از اصلاح‌گرهای املایی مانند ویراست‌من، اغلب به ازای هر واژه چندین واژه کاندیدا پیشنهاد می‌دهند. در این ارزیابی، تنها نخستین پیشنهاد هر سامانه، به عنوان واژه گزارش شده توسط سامانه برای هر واژه لحاظ شده و سایر پیشنهادها حتی در صورت صحت، نادیده گرفته شده‌اند.



## جدول و نتایج   


<br/>

### دادگان ذره‌بین

| الگوریتم | نرخ کشف کلمات نادرست | نرخ تصحیح کلمات خطا | نرخ تخریب کلمات صحیح  | نرخ صحت کشف خطاها  |
| -- | -- | -- | -- | -- |
| گوگل | **۰.۹۳۱۸** | **۰.۹۱۴** | ۰.۰۰۲۶ | ۰.۹۹۶۲ |
| نویسه۲ | ۰.۸۹۵۵ | ۰.۸۱۳۱ | ۰.۰۰۱۶ | ۰.۹۹۷۶ |
| پاک‌نویس | ۰.۸۷۷ | ۰.۷۸۸۹ | ۰.۰۳۱۷ | ۰.۹۵۵۸ |
| ویراست‌من | ۰.۸۳۴۲ | ۰.۸۰۲۸ | **۰.۰۰۰۶** | **۰.۹۹۹۱** |



<br/>

### دادگان PerSpellData 

| الگوریتم | نرخ کشف کلمات نادرست | نرخ تصحیح کلمات خطا | نرخ تخریب کلمات صحیح  | نرخ صحت کشف خطاها  |
| -- | -- | -- | -- | -- |
| پاک‌نویس | **۰.۹۴۵۵** | ۰.۸۲۴۵ | ۰.۰۱۶ | ۰.۸۳۹۶ |
| نویسه۲ | ۰.۹۱۹۶ | **۰.۸۳۵۸** | ۰.۰۰۳۴ | ۰.۹۵۹۴ |
| ویراست‌من | ۰.۷۹۵۲ | ۰.۶۵۰۸ | **۰.۰۰۰۸** | **۰.۹۸۹۲** |


<br/>

### دادگان شرق

| الگوریتم | نرخ کشف کلمات نادرست | نرخ تصحیح کلمات خطا | نرخ تخریب کلمات صحیح  | نرخ صحت کشف خطاها  |
| -- | -- | -- | -- | -- |
| نویسه۲ | **۰.۸۲۴۳** | **۰.۶۴۸۶** | ۰.۰۰۳۸ | ۰.۹۶۸۳ |
| پاک‌نویس | ۰.۷۵۶۸ | ۰.۵۸۱۱ | ۰.۰۲۹۵ | ۰.۷۸۵ |
| ویراست‌من | ۰.۶۰۳۶ | ۰.۴۸۲ | **۰.۰** | **۱.۰** |


<br/>

### دادگان نویسه - متن خبر

| الگوریتم | نرخ کشف کلمات نادرست | نرخ تصحیح کلمات خطا | نرخ تخریب کلمات صحیح  | نرخ صحت کشف خطاها  |
| -- | -- | -- | -- | -- |
| نویسه۲ | **۰.۸۳۸۶** | **۰.۷۳۶۷** | **۰.۰۰۳۷** | **۰.۹۸۳۲** |
| پاک‌نویس | ۰.۷۹۳۱ | ۰.۶۶۰۷ | ۰.۰۲۱۷ | ۰.۹۰۶۳ |


<br/>

### دادگان نویسه - عنوان خبر - ۵۳۹ جمله

| الگوریتم | نرخ کشف کلمات نادرست | نرخ تصحیح کلمات خطا | نرخ تخریب کلمات صحیح  | نرخ صحت کشف خطاها  |
| -- | -- | -- | -- | -- |
| نویسه۲ | **۰.۸۳۱۴** | **۰.۷۲۱۶** | **۰.۰۰۳** | **۰.۹۶۸** |
| پاک‌نویس | ۰.۷۸۴۳ | ۰.۶۷۰۶ | ۰.۲۲۸ | ۰.۷۹۲۱ |
| گوگل | ۰.۷۳۹۲ | ۰.۷۰۲ | ۰.۰۰۴۵ | ۰.۹۴۴۹ |
| ویراست‌من | ۰.۶ | ۰.۵ | ۰.۰۰۳۲ | ۰.۹۵۳۳ |



<br/>

### دادگان نویسه - عنوان خبر

| الگوریتم | نرخ کشف کلمات نادرست | نرخ تصحیح کلمات خطا | نرخ تخریب کلمات صحیح  | نرخ صحت کشف خطاها  |
| -- | -- | -- | -- | -- |
| نویسه۲ | **۰.۸۲۸** | **۰.۷۱۰۲** | **۰.۰۰۸۶** | **۰.۹۰۸۳** |
| پاک‌نویس | ۰.۷۸۰۲ | ۰.۶۴۵۸ | ۰.۰۲۳۵ | ۰.۷۷۷ |


</div>


# کد

## آماده‌سازی

[دانلود دادگان](https://drive.google.com/drive/folders/1JhU8JQI5bLiLbWdvDJqvDTJpRm_D-Xa0?usp=sharing)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Dadmatech/spell_checker/comparison

Mounted at /content/drive
/content/drive/MyDrive/Dadmatech/spell_checker/comparison


In [2]:
with open('./data/chars.txt') as f:
  chars = f.readlines()
char_dict = {c.split(', ')[0]:c.split(', ')[1][0] for c in chars}

def char_refinement(text):
  s = ''
  for i in text:
    if i in normal_chars:
      s += normal_chars[i]
    elif i in ['\u200c', '_', ' ']:
      s += i
    elif not i in special_chars:
      s += i
  return s

In [3]:
normal_chars = {}
for i in char_dict:
  if char_dict[i] != '\n':
    normal_chars[i] = char_dict[i]
# normal_chars

In [4]:
special_chars = []
for i in char_dict:
  if char_dict[i] == '\n':
    special_chars.append(i)
# special_chars

## خواندن فایل‌ها

In [5]:
def read_dataset (dataset_name, dataset_type):
  with open(f'data/{dataset_name}/{dataset_name}_{dataset_type}.txt') as f:
    data = f.readlines()
  return data

In [6]:
# zarebin

zarebin_wrongs = read_dataset('zarebin', 'wrongs')
zarebin_corrects = read_dataset('zarebin', 'corrects')
zarebin_google = read_dataset('zarebin', 'google')
zarebin_virastman = read_dataset('zarebin', 'virastman')
zarebin_paknevis = read_dataset('zarebin', 'paknevis')
zarebin_nevise = read_dataset('zarebin', 'nevise')

print(f'lengths: {len(zarebin_virastman)}, {len(zarebin_paknevis)}, {len(zarebin_nevise)}, {len(zarebin_google)}' )

lengths: 1033, 1033, 1033, 1033


In [7]:
# PerSpellData

PerSpellData_wrongs = read_dataset('PerSpellData', 'wrongs')
PerSpellData_corrects = read_dataset('PerSpellData', 'corrects')
PerSpellData_virastman = read_dataset('PerSpellData', 'virastman')
PerSpellData_paknevis = read_dataset('PerSpellData', 'paknevis')
PerSpellData_nevise = read_dataset('PerSpellData', 'nevise')

print(f'lengths: {len(PerSpellData_virastman)}, {len(PerSpellData_paknevis)}, {len(PerSpellData_nevise)}' )

lengths: 1127, 1127, 1127


In [8]:
# shargh

shargh_wrongs = read_dataset('shargh', 'wrongs')
shargh_corrects = read_dataset('shargh', 'corrects')
shargh_virastman = read_dataset('shargh', 'virastman')
shargh_paknevis = read_dataset('shargh', 'paknevis')
shargh_nevise = read_dataset('shargh', 'nevise')

print(f'lengths: {len(shargh_corrects)}, {len(shargh_wrongs)}, {len(shargh_virastman)}, {len(shargh_paknevis)}, {len(shargh_nevise)}')

lengths: 223, 223, 223, 223, 223


In [9]:
# nevise news 451

nevise_wrongs = read_dataset('nevise-news-451', 'wrongs')
nevise_corrects = read_dataset('nevise-news-451', 'corrects')
nevise_paknevis = read_dataset('nevise-news-451', 'paknevis')
nevise_nevise = read_dataset('nevise-news-451', 'nevise')

print(f'lengths: {len(nevise_paknevis)}, {len(nevise_nevise)}' )

lengths: 451, 451


In [10]:
# nevise news title 539

nevise_titles_wrongs = read_dataset('nevise-news-title-539', 'wrongs')
nevise_titles_corrects = read_dataset('nevise-news-title-539', 'corrects')
nevise_titles_google = read_dataset('nevise-news-title-539', 'google')
nevise_titles_virastman = read_dataset('nevise-news-title-539', 'virastman')
nevise_titles_paknevis = read_dataset('nevise-news-title-539', 'paknevis')
nevise_titles_nevise = read_dataset('nevise-news-title-539', 'nevise')
nevise_titles_nevise_v1 = read_dataset('nevise-news-title-539', 'nevise-v1')

print(f'lengths: {len(nevise_titles_google)}, {len(nevise_titles_paknevis)}, {len(nevise_titles_nevise)}')

lengths: 539, 539, 539


In [11]:
# nevise news title all

nevise_titles_all_wrongs = read_dataset('nevise-news-title-all', 'wrongs')
nevise_titles_all_corrects = read_dataset('nevise-news-title-all', 'corrects')
nevise_titles_all_paknevis = read_dataset('nevise-news-title-all', 'paknevis')
nevise_titles_all_nevise = read_dataset('nevise-news-title-all', 'nevise')

print(f'lengths: {len(nevise_titles_all_paknevis)}, {len(nevise_titles_all_nevise)}, {len(nevise_titles_all_corrects)}' )

lengths: 19421, 19421, 19421


## توابع ارزیابی

In [12]:
from nltk import edit_distance

In [13]:
def get_tok_info(index, w_tok, c_tokens, p_tokens, p_tokens_without_special, p_tokens_subwords, p_tokens_bi, verbose=False):
  string_file = ''
  output_info = {
      'wrong': False,
      'not_detected': False,
      'detected': False,
      'c2w': False,
      'w2c': False,
      'c2c': False,
  }
  if verbose:
    print('\n', w_tok)
  is_correct = pseudo_is_contain(query=c_tokens, key=w_tok, key_index=index)
  is_changed = not pseudo_is_contain(
      query=p_tokens, key=w_tok, key_index=index
      ) and not pseudo_is_contain(
          query=p_tokens_without_special, key=w_tok, key_index=index
      ) and not pseudo_is_contain(
          query=p_tokens_subwords, key=w_tok, key_index=index
      ) and not pseudo_is_contain(
          query=p_tokens_bi, key=w_tok, key_index=index
      ) 

  if not is_correct:
    output_info['wrong'] = True
    if verbose:
      print('wrong', w_tok)
    # string_file +=  f'wrong {w_tok}\n'

  if not is_correct and not is_changed:
    output_info['not_detected'] = True
    # string_file += f'not_detected {w_tok}\n'
    if verbose:
      print('not_detected', w_tok)

  if is_correct and is_changed:
    output_info['c2w'] = True
    if verbose:
      print('c2w', w_tok)
    string_file += f'c2w {w_tok}\n'

  if is_correct and not is_changed:
    output_info['c2c'] = True
    if verbose:
      print('c2c', w_tok)
    # string_file += f'c2c {w_tok}\n'

  if (not is_correct) and is_changed:
    output_info['detected'] = True
    if verbose:
      print('detected', w_tok)
    #w_detected += 1
    # string_file += f'detected {w_tok}\n'

    c_paired_tok = get_paired_token(query=c_tokens, key=w_tok, key_index=index)
    p_paired_tok = get_paired_token(query=p_tokens, key=w_tok, key_index=index)
    p_paired_tok_without_special = get_paired_token(query=p_tokens_without_special, key=w_tok, key_index=index)
    p_paired_tok_subwords = get_paired_token(query=p_tokens_subwords, key=w_tok, key_index=index)
    p_paired_tok_bi = get_paired_token(query=p_tokens_bi, key=w_tok, key_index=index)
    if verbose:
      print(c_paired_tok, p_paired_tok, p_paired_tok_without_special, p_paired_tok_subwords)
    if is_soft_equal(
        c_paired_tok, p_paired_tok
        ) or is_soft_equal(
            c_paired_tok, p_paired_tok_without_special
        ) or is_soft_equal(
            c_paired_tok, p_paired_tok_subwords
        ) or is_soft_equal(
            c_paired_tok, p_paired_tok_bi
        ):    
      output_info['w2c'] = True
      if verbose:
        print('w2c', w_tok)
      # string_file += f'w2c {w_tok}\n'
      
  return output_info, string_file


def get_paired_token(query, key, key_index, r=4):
  most_similar = ''
  most_similar_score = len(query)
  for query_token in query[max(0, key_index - r): min(len(query), key_index + r)]:
    distance = edit_distance(query_token, key)
    if most_similar_score > distance:
      most_similar = query_token
      most_similar_score = distance
  return most_similar

def is_soft_equal(c_token, p_token):
  return c_token == p_token or c_token == remove_special_chars(p_token)


def remove_special_chars(text):
  for special_ch in special_chars:
    text = text.replace(special_ch, '')
  return text

def replace_halfspaces(text, replace_with=' '):
  text = text.replace('\\u200c', replace_with)
  return text.replace('\u200c', replace_with)


def pseudo_is_contain(query, key, key_index, r=4):
  return key in query[max(0, key_index - r): min(len(query), key_index + r)]


def words_halfspaces (list_of_words):
  list_of_words = [word.split('‌') for word in list_of_words]
  return [sub for words in list_of_words for sub in words]

In [14]:
def update_eval_params(eval_params, info):
  for i in info:
    eval_params[i] += info[i]
  return eval_params

def best_info (info1, info2):
  if info2['c2c'] or info2['w2c']:
    return info2, 2
  return info1, 1

def eval(wrong_sentence, correct_sentence, predicted_sentence, verbose=False):
    eval_params = {
          'wrong': 0,
          'not_detected': 0,
          'detected': 0,
          'c2w': 0,
          'w2c': 0,
          'c2c': 0,    
    }
    string_file_all = ''

    w_tokens = char_refinement(wrong_sentence).split()
    c_tokens = char_refinement(correct_sentence).split()
    p_tokens = char_refinement(predicted_sentence).split()

    w_tokens = [str(int(tok)) if tok.isdigit() else tok for tok in w_tokens]
    c_tokens = [str(int(tok)) if tok.isdigit() else tok for tok in c_tokens]
    p_tokens = [str(int(tok)) if tok.isdigit() else tok for tok in p_tokens]
    
    w_tokens_bi = [f'{w1}{w2}' for w1, w2 in zip(w_tokens, w_tokens[1:])]
    c_tokens_bi = [f'{c1}{c2}' for c1, c2 in zip(c_tokens, c_tokens[1:])]

    p_tokens = [p.replace('\\u200c', '\u200c') for p in p_tokens]
    p_tokens_special_removed = [remove_special_chars(p) for p in p_tokens]
    p_tokens_subwords = words_halfspaces(p_tokens)
    p_tokens_bi = [f'{p1}{p2}' for p1, p2 in zip(p_tokens, p_tokens[1:])]
    c_tokens = [remove_special_chars(c) for c in c_tokens]
    w_tokens = [remove_special_chars(w) for w in w_tokens]
    if verbose:
      print('in', w_tokens)
      print('correct', c_tokens)
      print('origin', p_tokens)
      print('removed', p_tokens_special_removed)
      print('sub', p_tokens_subwords)
      print('bi', p_tokens_bi)
    double_continue_flag = False
    for index, w_t in enumerate(w_tokens):
      if double_continue_flag:
        double_continue_flag = False
        continue
      which = 1
      info, string_file = get_tok_info(index, w_t, c_tokens, p_tokens, p_tokens_special_removed, p_tokens_subwords, p_tokens_bi)

      if index < len(w_tokens)-1:
        w_tok = w_tokens_bi[index]

        info_bi, string_file_bi = get_tok_info(index, w_tok, c_tokens_bi, p_tokens, p_tokens_special_removed, p_tokens_subwords, len(p_tokens_bi)*[' '])
        
        info, which = best_info(info, info_bi)
        if which == 2:
          if string_file_bi != '' :
            string_file_all += '** bi\n **' + string_file_bi
          double_continue_flag = True


      eval_params = update_eval_params(eval_params, info)
      if which == 1 :
        if string_file != '' :
          string_file_all += '** normal\n **' + string_file
    # string_file_all += f'{eval_params}'
    # print(eval_params)
    return eval_params, string_file_all

def write_log(string_to_file, names):
  with open(f'../logs/{names[0]}_{names[1]}_log.txt', 'w') as f:
    f.write(string_to_file)


def eval_spell_checker (wrongs, corrects, predicts, names):
  eval_params = {
          'wrong': 0,
          'not_detected': 0,
          'detected': 0,
          'c2w': 0,
          'w2c': 0,
          'c2c': 0,    
  }
  string_file = ''
  for index, (wrong_sentence, correct_sentence, predicted_sentence) in enumerate(zip(wrongs, corrects, predicts)):
    sentence_eval_params, string_file_sentence = eval(wrong_sentence, correct_sentence, predicted_sentence)
    if string_file_sentence != '':
      string_file += f'\n{index}\n'
      string_file += wrong_sentence +'\n'
      string_file += correct_sentence +'\n'
      string_file += predicted_sentence + '\n'
    
    eval_params = update_eval_params(eval_params, sentence_eval_params)
    string_file += string_file_sentence
  # print(eval_params)
  write_log(string_file, names)
  print()
  w2w = eval_params['detected'] - eval_params['w2c']
  wd_rate = round(eval_params['detected']/eval_params['wrong'], 4)
  w2c_rate = round(eval_params['w2c']/eval_params['wrong'], 4)
  c2w_rate = round(eval_params['c2w']/(eval_params['c2w'] + eval_params['c2c']), 4)
  precision = round((eval_params['w2c'] + w2w)/(eval_params['c2w'] + eval_params['w2c'] + w2w) , 4)
  print(f'{names[1]} on {names[0]}')
  print(f'wrong detection rate:{wd_rate} \twrong correction rate:{w2c_rate} \tcorrect to wrong rate:{c2w_rate} \tprecision:{precision}')
  print(eval_params)
  return eval_params

In [15]:
# tiny test 

eval_nevise_titles_nevise = eval_spell_checker (nevise_titles_wrongs[:10], nevise_titles_corrects[:10], nevise_titles_nevise[:10], ['Nevise news titles', 'Nevise2'])


Nevise2 on Nevise news titles
wrong detection rate:1.0 	wrong correction rate:0.7778 	correct to wrong rate:0.0125 	precision:0.9
{'wrong': 9, 'not_detected': 0, 'detected': 9, 'c2w': 1, 'w2c': 7, 'c2c': 79}


## اجرای ارزیابی

In [16]:
# Result on Zarebin
eval_zarebin_google = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_google, ['Zarebin', 'Google'])
eval_zarebin_virastman = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_virastman, ['Zarebin', 'Virastman'])
eval_zarebin_paknevis = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_paknevis, ['Zarebin', 'Paknevis'])
eval_zarebin_nevise = eval_spell_checker (zarebin_wrongs, zarebin_corrects, zarebin_nevise, ['Zarebin', 'Nevise2'])


Google on Zarebin
wrong detection rate:0.9318 	wrong correction rate:0.914 	correct to wrong rate:0.0026 	precision:0.9962
{'wrong': 1407, 'not_detected': 96, 'detected': 1311, 'c2w': 5, 'w2c': 1286, 'c2c': 1887}

Virastman on Zarebin
wrong detection rate:0.8342 	wrong correction rate:0.8028 	correct to wrong rate:0.0006 	precision:0.9991
{'wrong': 1405, 'not_detected': 233, 'detected': 1172, 'c2w': 1, 'w2c': 1128, 'c2c': 1780}

Paknevis on Zarebin
wrong detection rate:0.877 	wrong correction rate:0.7889 	correct to wrong rate:0.0317 	precision:0.9558
{'wrong': 1407, 'not_detected': 173, 'detected': 1234, 'c2w': 57, 'w2c': 1110, 'c2c': 1739}

Nevise2 on Zarebin
wrong detection rate:0.8955 	wrong correction rate:0.8131 	correct to wrong rate:0.0016 	precision:0.9976
{'wrong': 1407, 'not_detected': 147, 'detected': 1260, 'c2w': 3, 'w2c': 1144, 'c2c': 1833}


In [17]:
# Result on PerSpellData

eval_perspell_virastman = eval_spell_checker (PerSpellData_wrongs, PerSpellData_corrects, PerSpellData_virastman, ['PerSpellData', 'Virastman'])
eval_perspell_paknevis = eval_spell_checker (PerSpellData_wrongs, PerSpellData_corrects, PerSpellData_paknevis, ['PerSpellData', 'Paknevis'])
eval_perspell_nevise = eval_spell_checker (PerSpellData_wrongs, PerSpellData_corrects, PerSpellData_nevise, ['PerSpellData', 'Nevise2'])


Virastman on PerSpellData
wrong detection rate:0.7952 	wrong correction rate:0.6508 	correct to wrong rate:0.0008 	precision:0.9892
{'wrong': 1157, 'not_detected': 237, 'detected': 920, 'c2w': 10, 'w2c': 753, 'c2c': 13252}

Paknevis on PerSpellData
wrong detection rate:0.9455 	wrong correction rate:0.8245 	correct to wrong rate:0.016 	precision:0.8396
{'wrong': 1157, 'not_detected': 63, 'detected': 1094, 'c2w': 209, 'w2c': 954, 'c2c': 12848}

Nevise2 on PerSpellData
wrong detection rate:0.9196 	wrong correction rate:0.8358 	correct to wrong rate:0.0034 	precision:0.9594
{'wrong': 1157, 'not_detected': 93, 'detected': 1064, 'c2w': 45, 'w2c': 967, 'c2c': 13321}


In [18]:
# Result on Shargh

eval_shargh_virastman = eval_spell_checker (shargh_wrongs, shargh_corrects, shargh_virastman, ['Shargh', 'Virastman'])
eval_shargh_paknevis = eval_spell_checker (shargh_wrongs, shargh_corrects, shargh_paknevis, ['Shargh', 'Paknevis'])
eval_shargh_nevise = eval_spell_checker (shargh_wrongs, shargh_corrects, shargh_nevise, ['Shargh', 'Nevise2'])


Virastman on Shargh
wrong detection rate:0.6036 	wrong correction rate:0.482 	correct to wrong rate:0.0 	precision:1.0
{'wrong': 222, 'not_detected': 88, 'detected': 134, 'c2w': 0, 'w2c': 107, 'c2c': 1575}

Paknevis on Shargh
wrong detection rate:0.7568 	wrong correction rate:0.5811 	correct to wrong rate:0.0295 	precision:0.785
{'wrong': 222, 'not_detected': 54, 'detected': 168, 'c2w': 46, 'w2c': 129, 'c2c': 1515}

Nevise2 on Shargh
wrong detection rate:0.8243 	wrong correction rate:0.6486 	correct to wrong rate:0.0038 	precision:0.9683
{'wrong': 222, 'not_detected': 39, 'detected': 183, 'c2w': 6, 'w2c': 144, 'c2c': 1579}


In [19]:
# Result on Nevise 

eval_nevise_paknevis = eval_spell_checker (nevise_wrongs, nevise_corrects, nevise_paknevis, ['Nevise', 'Paknevis'])
eval_nevise_nevise = eval_spell_checker (nevise_wrongs, nevise_corrects, nevise_nevise, ['Nevise', 'Nevise2'])


Paknevis on Nevise
wrong detection rate:0.7931 	wrong correction rate:0.6607 	correct to wrong rate:0.0217 	precision:0.9063
{'wrong': 2305, 'not_detected': 477, 'detected': 1828, 'c2w': 189, 'w2c': 1523, 'c2c': 8509}

Nevise2 on Nevise
wrong detection rate:0.8386 	wrong correction rate:0.7367 	correct to wrong rate:0.0037 	precision:0.9832
{'wrong': 2305, 'not_detected': 372, 'detected': 1933, 'c2w': 33, 'w2c': 1698, 'c2c': 8827}


In [20]:
# Result on nevisee news title 539

eval_nevise_titles_google = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_google, ['Nevise-news-titles-539', 'Google'])
eval_nevise_titles_virastman = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_virastman, ['Nevise-news-titles-539', 'Virastman'])
eval_nevise_titles_paknevis = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_paknevis, ['Nevise-news-titles-539', 'Paknevis'])
eval_nevise_titles_nevise = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_nevise, ['Nevise-news-titles-539', 'Nevise2'])
eval_nevise_titles_nevise_v1 = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_nevise_v1, ['nevise-news-title-539', 'nevise-v1'])



Google on Nevise-news-titles-539
wrong detection rate:0.7392 	wrong correction rate:0.702 	correct to wrong rate:0.0045 	precision:0.9449
{'wrong': 510, 'not_detected': 133, 'detected': 377, 'c2w': 22, 'w2c': 358, 'c2c': 4848}

Virastman on Nevise-news-titles-539
wrong detection rate:0.6 	wrong correction rate:0.5 	correct to wrong rate:0.0032 	precision:0.9533
{'wrong': 510, 'not_detected': 204, 'detected': 306, 'c2w': 15, 'w2c': 255, 'c2c': 4635}

Paknevis on Nevise-news-titles-539
wrong detection rate:0.7843 	wrong correction rate:0.6706 	correct to wrong rate:0.0228 	precision:0.7921
{'wrong': 510, 'not_detected': 110, 'detected': 400, 'c2w': 105, 'w2c': 342, 'c2c': 4497}

Nevise2 on Nevise-news-titles-539
wrong detection rate:0.8314 	wrong correction rate:0.7216 	correct to wrong rate:0.003 	precision:0.968
{'wrong': 510, 'not_detected': 86, 'detected': 424, 'c2w': 14, 'w2c': 368, 'c2c': 4691}

nevise-v1 on nevise-news-title-539
wrong detection rate:0.7647 	wrong correction rate:

In [21]:
# Result on nevisee news title 539

eval_nevise_titles_google = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_google, ['Nevise-news-titles-539', 'Google'])
eval_nevise_titles_virastman = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_virastman, ['Nevise-news-titles-539', 'Virastman'])
eval_nevise_titles_paknevis = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_paknevis, ['Nevise-news-titles-539', 'Paknevis'])
eval_nevise_titles_nevise = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_nevise, ['Nevise-news-titles-539', 'Nevise2'])
eval_nevise_titles_nevise_v1 = eval_spell_checker (nevise_titles_wrongs, nevise_titles_corrects, nevise_titles_nevise_v1, ['nevise-news-title-539', 'nevise-v1'])



Google on Nevise-news-titles-539
wrong detection rate:0.7392 	wrong correction rate:0.702 	correct to wrong rate:0.0045 	precision:0.9449
{'wrong': 510, 'not_detected': 133, 'detected': 377, 'c2w': 22, 'w2c': 358, 'c2c': 4848}

Virastman on Nevise-news-titles-539
wrong detection rate:0.6 	wrong correction rate:0.5 	correct to wrong rate:0.0032 	precision:0.9533
{'wrong': 510, 'not_detected': 204, 'detected': 306, 'c2w': 15, 'w2c': 255, 'c2c': 4635}

Paknevis on Nevise-news-titles-539
wrong detection rate:0.7843 	wrong correction rate:0.6706 	correct to wrong rate:0.0228 	precision:0.7921
{'wrong': 510, 'not_detected': 110, 'detected': 400, 'c2w': 105, 'w2c': 342, 'c2c': 4497}

Nevise2 on Nevise-news-titles-539
wrong detection rate:0.8314 	wrong correction rate:0.7216 	correct to wrong rate:0.003 	precision:0.968
{'wrong': 510, 'not_detected': 86, 'detected': 424, 'c2w': 14, 'w2c': 368, 'c2c': 4691}

nevise-v1 on nevise-news-title-539
wrong detection rate:0.7647 	wrong correction rate:

In [22]:
# Result on nevise news title

eval_nevise_titles_all_paknevis = eval_spell_checker (nevise_titles_all_wrongs, nevise_titles_all_corrects, nevise_titles_all_paknevis, ['Nevise-news-titles', 'Paknevis'])
eval_nevise_titles_all_nevise = eval_spell_checker (nevise_titles_all_wrongs, nevise_titles_all_corrects, nevise_titles_all_nevise, ['Nevise-news-titles', 'Nevise2'])


Paknevis on Nevise-news-titles
wrong detection rate:0.7802 	wrong correction rate:0.6458 	correct to wrong rate:0.0235 	precision:0.777
{'wrong': 18365, 'not_detected': 4037, 'detected': 14328, 'c2w': 4112, 'w2c': 11861, 'c2c': 170557}

Nevise2 on Nevise-news-titles
wrong detection rate:0.828 	wrong correction rate:0.7102 	correct to wrong rate:0.0086 	precision:0.9083
{'wrong': 18368, 'not_detected': 3160, 'detected': 15208, 'c2w': 1536, 'w2c': 13045, 'c2c': 176100}
